# LangChain Agents - Financial Analysis with Gemini

Complete building production-ready agents with memory, middleware, and streaming

## Overview 
This notebook covers:  
- Short-term Memory: Persist conversation state with SQLite
- Built-in Middleware: Production partterns (summarization, limits, PII detection, todo tracking)
- Structured Output: Type-safe agent responses with Pydantic
- Streaming Modes: Real-time updates (messages, updates, values)

## Setup
Initialize model and tools for financial analysis

In [ ]:
import os 
from dotenv import load_dotenv